# 実行環境の作成

## Google Cloud Storage

下記コードでGCPに接続

In [1]:
from google.colab import auth
auth.authenticate_user()

認証に成功したらgcsfuseをインストール

In [2]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt-get -y -q update
!apt-get -y -q install gcsfuse

deb http://packages.cloud.google.com/apt gcsfuse-bionic main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  79281      0 --:--:-- --:--:-- --:--:-- 79281
OK
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:4 http://packages.cloud.google.com/apt gcsfuse-bionic InRelease [5,391 B]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/lin

バケット「statistics-hyogo」をディレクトリ「statistics-hyogo」にマウント

In [3]:
! mkdir -p statistics-hyogo
! gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 statistics-hyogo statistics-hyogo

2022/09/16 07:50:05.756427 Start gcsfuse/0.41.6 (Go version go1.18.4) for app "" using mount point: /content/statistics-hyogo
2022/09/16 07:50:05.772522 Opening GCS connection...
2022/09/16 07:50:07.194689 Mounting file system "statistics-hyogo"...
2022/09/16 07:50:07.195464 File system has been successfully mounted.


## ESTAT_APP_ID

In [4]:
ESTAT_APPID = '724e5b90772a3e9289f41a253e4e7e32438f4fff'

## pythonのライブラリ追加

In [5]:
import urllib.parse
import urllib.request
import json
import pandas as pd

# estatの統計情報を取得

'statsDataId'に統計表コードを指定

In [6]:
statsDataId = '0000010101'

メタ情報を取得する関数を作成

In [7]:
def get_estat_meta(statsDataId):
  # リクエストパラメータ
  p={}
  p['statsDataId'] = statsDataId
  p['appId'] = ESTAT_APPID

  url = 'http://api.e-stat.go.jp/rest/3.0/app/json/getMetaInfo?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

meta = get_estat_meta(statsDataId)
# print(meta)

## DataFrameの作成

メタ情報をDataFrameに整形する

In [8]:
def get_cards_dataframe(statsDataId):
  # メタ情報の取得
  meta= get_estat_meta(statsDataId)
 
  # STAT_NAME
  STAT_NAME = meta['GET_META_INFO']['METADATA_INF']['TABLE_INF']['STAT_NAME']['$']

  # CLASS_OBJ
  CLASS_OBJ = meta['GET_META_INFO']['METADATA_INF']['CLASS_INF']['CLASS_OBJ']

  # cat01
  cat01 = next((d for d in CLASS_OBJ if d['@id'] == 'cat01'), None)['CLASS']

  # DataFrame
  df = pd.json_normalize(cat01)[['@code','@name']]
  
  # 統計表情報の追加
  df['statsDataId'] = statsDataId
  df['statsDataName'] = STAT_NAME

  # 列名の変更
  columns = {'@code':'categoryCode','@name':'categoryName'}
  df= df.rename(columns=columns)

  # categoryNameから不要な情報（categoryCode）を削除
  df['categoryName'] = df.apply(lambda x: x['categoryName'].replace(x['categoryCode']+'_', ''), 1)
  
  
  # 並べ替え
  df = df.reindex(columns=['statsDataId', 'statsDataName', 'categoryCode','categoryName'])
  
  return df


# print(len(category_codes))

In [9]:
df = get_cards_dataframe(statsDataId)
df

,statsDataId,statsDataName,categoryCode,categoryName
0,0000010101,社会・人口統計体系,A1101,総人口
1,0000010101,社会・人口統計体系,A110101,総人口（男）
2,0000010101,社会・人口統計体系,A110102,総人口（女）
3,0000010101,社会・人口統計体系,A1102,日本人人口
4,0000010101,社会・人口統計体系,A110201,日本人人口（男）
...,...,...,...,...
563,0000010101,社会・人口統計体系,A850205,父子世帯数（55歳以上）
564,0000010101,社会・人口統計体系,A9101,婚姻件数
565,0000010101,社会・人口統計体系,A9111,平均婚姻年齢（初婚の夫）
566,0000010101,社会・人口統計体系,A9112,平均婚姻年齢（初婚の妻）


## 統計カード管理情報をSpreadSheetで加工

統計カードの管理情報を SpreadSheetで管理する

前セルで作業したDataFrameをcsvでコピーするなどして作業すること


[statistics-hyogo/統計カード管理](https://docs.google.com/spreadsheets/d/1mAv2gx9khNOty_ILu3aJjJz0uoRkARzwoaG4VBUSNG4/edit#gid=0)

## BigQueryへ保存

ライブラリのインストール

In [10]:
!pip install --upgrade -q gspread

SpreadSheetの認証

In [11]:
from google.colab import auth
from google.auth import default
import gspread

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

worksheetを読み込んでDataFrameに変換

In [ ]:
# スプレッドシートの情報
ss_id = "1mAv2gx9khNOty_ILu3aJjJz0uoRkARzwoaG4VBUSNG4"
workbook = gc.open_by_key(ss_id)
worksheet = workbook.worksheet("シート1")

# DataFrameに格納
dic = worksheet.get_all_records(head=1)
df = pd.DataFrame(dic)
df

pandas-gbqのインストール

In [ ]:
!pip install pandas-gbq

DataFrameをBigQueryへ保存

In [ ]:
import pandas as pd

# プロジェクトの定義
project_id='primal-buttress-342908'
dataset_id = 'contents'
table_id='cards'

# データフレームの内容をBigQueryのテーブルに追加
df.to_gbq( f'{dataset_id}.{table_id}', project_id=project_id, if_exists="replace")